In [2]:
import scanpy as sc
import scgen # pip install git+https://github.com/theislab/scgen.git
import re
# import numpy as np

In [3]:
def get_clono_features(features):
    patterns = [
        "TRAV", "TRAJ", "TRBV", "TRBJ", "TRDV", "TRDJ", "TRGV", "TRGJ",
        "IGKV", "IGKJ", "IGLV", "IGLJ", "IGHV", "IGHJ"
    ]
    additional_features = ["XIST", "RPS4Y1", "RPS4Y2", "MT"]

    clono_features = []
    
    for pattern in patterns:
        clono_features.extend([i for i, feature in enumerate(features) if re.search(pattern, feature)])
    
    clono_features.extend([i for i, feature in enumerate(features) if feature in additional_features])
    
    clono_features = np.unique(clono_features).tolist()
    
    return clono_features

In [7]:
adata = sc.read_h5ad("data/raw_counts_adata.h5ad")
adata

/home/bench-user/.apps/conda/envs/scgen/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/bench-user/.apps/conda/envs/scgen/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 25066 × 15369
    obs: 'cell_id', 'nCount_RNA', 'nFeature_RNA', 'patient_id', 'percent_mito', 'Response_3m', 'sample_source'
    var: 'gene_name'

In [5]:
import numpy as np

In [ ]:
rm_gene_indices = get_clono_features(adata.var.gene_name.tolist())
mask = np.ones(adata.X.shape[1], dtype=bool) # create a boolean mask for the genes to keep
mask[rm_gene_indices] = False

Tcell_data = adata[:, mask]

In [10]:
adata.var.gene_name

0        AP006222.2
1            FAM87B
2         LINC00115
3            FAM41C
4            SAMD11
            ...    
15364       MT-ND4L
15365        MT-ND4
15366        MT-ND5
15367        MT-ND6
15368        MT-CYB
Name: gene_name, Length: 15369, dtype: object

In [11]:
remove_gene_indices = get_clono_features(adata.var.gene_name.tolist())
mask = np.ones(adata.shape[1], dtype=bool)
mask[remove_gene_indices] = False

adata = adata[:,mask]

print("dimension after removing selected genes:", adata.X.shape) # remove 150 genes

dimension after removing selected genes: (25066, 15219)


In [13]:
adata.obs["Response_3m"].value_counts()

Response_3m
OR    12220
NR    11906
CR      940
Name: count, dtype: int64